# Install required packages

In [1]:
!pip install pandas nltk textblob python-docx
!python -m textblob.download_corpora

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 1.2 MB/s eta 0:00:00
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


In [2]:
import pandas as pd
import nltk
from textblob import TextBlob
from docx import Document
from collections import defaultdict
from google.colab import files

#  Download NLTK resources

In [3]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
# Directly read the uploaded file from the Colab filesystem
df = pd.read_csv("/content/Stock Market 6thJuly.csv")

# Optional: preview the data
df.head()

,Ticker,Title,Summary,URL,Original Published Time,Datetime,News Age (Days),News Time,Current Price,1 Hour Ago,4 Hours Ago,Previous Day Close,7 Days Ago,1 Hour Ago % Change,4 Hours Ago % Change,Previous Day Close % Change,7 Days Ago % Change,Headline Sentiment,Summary Sentiment
0,RYDE,Ryde Partners With Tencent Cloud For Enhanced ...,"Ryde Group Ltd. (AMEX:RYDE), a Singapore-based...",https://finance.yahoo.com/news/ryde-partners-t...,"Fri, 20 Jun 2025 10:34:31 +0000",2025-06-20 06:34:31-04:00,16,2025-06-20 06:34:31-04:00,0.41,0.41,0.41,NaN,0.18,0.0,0.0,NaN,127.78,0.0,0.18
1,RYDE,Tencent Cloud Empowers Ryde to Elevate In-App ...,"Tencent Cloud, the cloud business of global le...",https://finance.yahoo.com/news/tencent-cloud-e...,"Fri, 20 Jun 2025 09:00:00 +0000",2025-06-20 05:00:00-04:00,16,2025-06-20 05:00:00-04:00,0.41,0.41,0.41,NaN,0.18,0.0,0.0,NaN,127.78,0.0,0.00
2,RYDE,Ryde Bets Big on EVs: Acquires 40% Stake in At...,"SINGAPORE, SG / ACCESS Newswire / June 18, 202...",https://finance.yahoo.com/news/ryde-bets-big-e...,"Wed, 18 Jun 2025 09:00:00 +0000",2025-06-18 05:00:00-04:00,18,2025-06-18 05:00:00-04:00,0.36,0.36,0.36,0.18,0.21,0.0,0.0,100.00,71.43,0.0,0.00
3,NFLX,"Netflix Q2 Results Expected to Beat Guidance, ...",Netflix (NFLX) is expected to deliver Q2 resul...,https://finance.yahoo.com/news/netflix-q2-resu...,"Thu, 03 Jul 2025 11:43:05 +0000",2025-07-03 07:43:05-04:00,3,2025-07-03 07:43:05-04:00,1296.34,1296.34,1296.34,1293.00,1292.52,0.0,0.0,0.26,0.30,-0.1,0.00
4,NFLX,Trump’s Vietnam Trade Deal Is All About China....,"Tax megabill set for final vote, Tesla sees wo...",https://finance.yahoo.com/m/a34f8abd-7ae4-3dbd...,"Thu, 03 Jul 2025 10:52:00 +0000",2025-07-03 06:52:00-04:00,3,2025-07-03 06:52:00-04:00,1296.34,1296.34,1296.34,1293.00,1292.52,0.0,0.0,0.26,0.30,0.0,-0.27


In [7]:
# Define known financial keyword categories
KEYWORD_CATEGORIES = {
    "earnings": [
        "earnings", "profit", "profits", "revenue", "sales", "income",
        "loss", "losses", "guidance", "forecast", "EPS", "net income", "beat estimates",
        "missed estimates", "record profits", "quarterly results", "financial results", "operating margin"
    ],
    "merger": [
        "merger", "merging", "combine", "merge talks", "consolidation",
        "merger agreement", "merger approval", "merger blocked"
    ],
    "acquisition": [
        "acquisition", "acquire", "acquired", "buyout", "purchase", "takeover",
        "deal signed", "deal announced", "investment", "stake purchase", "ownership increase"
    ],
    "stimulus": [
        "stimulus", "government support", "funding", "bailout", "subsidy",
        "relief package", "stimulus package", "federal support", "tax credit"
    ],
    "FDA approval": [
        "FDA", "approval", "approved", "clinical", "clinical trial", "drug trial",
        "vaccine", "authorized", "emergency approval", "phase 3 trial", "trial success"
    ],
    "partnership": [
        "partnership", "collaboration", "joint venture", "strategic alliance",
        "deal partnership", "cooperation agreement", "multi-year contract"
    ],
    "product launch": [
        "launch", "introduced", "released", "new product", "rollout", "unveiled",
        "prototype", "product update", "new version", "flagship product"
    ],
    "dividend": [
        "dividend", "payout", "shareholder return", "buyback",
        "dividend increase", "dividend cut", "special dividend", "stock repurchase"
    ],
    "lawsuit": [
        "lawsuit", "legal case", "settlement", "court", "regulatory action",
        "antitrust", "fined", "penalty", "investigation", "regulator probe"
    ],
    "price target": [
        "price target", "upgrade", "downgrade", "analyst rating", "valuation",
        "buy rating", "sell rating", "neutral rating", "target raised", "target cut"
    ],
    "supply chain": [
        "supply chain", "logistics", "shipment", "delay", "inventory",
        "supply shortage", "backlog", "transportation issue", "production halt"
    ],
    "expansion": [
        "expansion", "growth plan", "new facility", "factory", "headquarters",
        "hiring", "job cuts", "layoffs", "new office", "opening new plant", "capacity increase"
    ],
    "market performance": [
        "stock surge", "stock drop", "market share", "benchmark", "index",
        "outperform", "underperform", "record high", "record low", "volatility", "sell-off", "rally"
    ],
    "technology": [
        "AI", "machine learning", "cloud", "software", "chip", "semiconductor",
        "innovation", "R&D", "5G", "blockchain", "cybersecurity", "patent approval"
    ],
    "economic outlook": [
        "inflation", "interest rates", "GDP", "unemployment", "recession",
        "economic slowdown", "rate hike", "rate cut", "consumer confidence"
    ],
    "environment": [
        "sustainability", "carbon", "green energy", "renewable", "ESG",
        "climate change", "solar project", "wind energy", "emission reduction"
    ],
    "crypto": [
        "bitcoin", "crypto", "cryptocurrency", "token", "blockchain",
        "NFT", "digital asset", "crypto regulation"
    ],
    "bankruptcy": [
        "bankruptcy", "insolvency", "debt restructuring", "chapter 11", "liquidation", "creditors"
    ]
}

# Detect known keywords in a text
def detect_keywords(text):
    detected = []
    for category, words in KEYWORD_CATEGORIES.items():
        for word in words:
            if word.lower() in text.lower():
                detected.append(category)
                break  # One match per category is enough
    return detected or ['Other']

# Storage dictionary
company_keywords = defaultdict(lambda: {'positive': [], 'negative': []})

# 🔍 Process each row to extract keywords & sentiment
for _, row in df.iterrows():
    ticker = row['Ticker']
    summary = str(row['Summary'])  # Ensure it's a string

    blob = TextBlob(summary)
    sentiment = 'positive' if blob.sentiment.polarity >= 0 else 'negative'

    detected = detect_keywords(summary)

    company_keywords[ticker][sentiment].extend(detected)

for ticker in company_keywords:
    for sentiment in ['positive', 'negative']:
        keywords = list(set(company_keywords[ticker][sentiment]))
        company_keywords[ticker][sentiment] = sorted(keywords)

# Create the Word document
doc = Document()
doc.add_heading('Company Progress Keyword Summary', 0)

for ticker, sentiment_data in sorted(company_keywords.items()):
    doc.add_heading(ticker, level=1)

    pos_keywords = ', '.join(sentiment_data['positive']) or "No data"
    neg_keywords = ', '.join(sentiment_data['negative']) or "No data"

    doc.add_paragraph(f"✅ Positive Keywords: {pos_keywords}")
    doc.add_paragraph(f"❌ Negative Keywords: {neg_keywords}")
    doc.add_paragraph("")  # Spacer

#  Save and download the DOCX

In [8]:
output_file = "Company_Progress_Summary.docx"
doc.save(output_file)
files.download(output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>